In [1]:
#加载云盘
from google.colab import drive
drive.mount('/content/drive')
#改变工作目录
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/DF/clound/')
!ls

#过12个小时 空间就会被自动清理
n = len(os.listdir('trainImg/'))
if n > 10000: 
    print("train有图像{}张".format(n))
else:
    os.chdir('/content/drive/My Drive/Colab Notebooks/DF/clound/trainImg')
    !ls
    !unzip -o Train.zip
    os.chdir('/content/drive/My Drive/Colab Notebooks/DF/clound/')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
clound.ipynb	    trainImg	     train_single.tfrecords
submit_example.csv  Train_label.csv
train有图像10669张


In [0]:
# os.chdir('/content/drive/My Drive/')
# !ls

In [0]:
# !find -maxdepth 1 -name "*.png" -exec rm -f {} \;
# !ls

In [4]:
import tensorflow as tf
import numpy as np
import keras
from keras import backend as K
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, Concatenate, Add
from keras.layers import BatchNormalization, Activation, Input, SeparableConv2D
from keras.models import Model, Sequential, load_model 
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler

Train_Img_Dir = 'trainImg/'
Train_Label_Path = 'Train_label.csv'
Test_Img_Dir = 'testImg/'
TS_TFRecords_File = 'train_single.tfrecords' #单标签的图片tfrecoard
TM_TFRecords_File = 'train_multi.tfrecords' #多标签的图片tfrecoard
submit_File = 'submit_example.csv'

X_shape = [500, 500, 3] # 预处理后图像的大小
EPOCH = 100 # 训练多少轮
Batch_size = 128  # 训练的batch size

# tf.enable_eager_execution() #调试用


Using TensorFlow backend.


In [0]:
def read_image(name, type):
    """读图片  
    name: 图片名
    type: ‘train' 'test'
    return: Image类型
    """
    if type == 'train':
        path = os.path.join(Train_Img_Dir, name)
    elif type == 'test': 
        path = os.path.join(Test_Img_Dir, name)

    img = Image.open(path) # img.mode = 'RGB'
    if img.mode != 'RGB':
        img = img.convert("RGB") #读取图片的过程中如果遇到非'RGB'就转换格式
    # plt.imshow(img)  
    # plt.show()
    return img 

class DatasetGenerator:
    def __init__(self):
        self.tls = self._get_train_label() #获取训练图片名列表
        #获取单标签的训练图片名和label列表
        self.train_single_labels = self.tls[self.tls.apply(lambda x: len(x['Code'])<=2, axis=1)] 
        self.train_single_num = len(self.train_single_labels) #图片总数
        #获取多标签的训练图片名和label列表(label长度2个以上)
        self.train_multi_labels = self.tls[self.tls.apply(lambda x: len(x['Code'])>2, axis=1)]  
        self.train_multi_num = len(self.train_multi_labels) #图片总数
        #单标签 标签类别数目，也是网络最后一层的单元数目
        self.single_class_num = len(self.train_single_labels['Code'].unique()) 

    def write2TFRecoard(self, type, start=0, end=None, option=None):
        """
        #把图片生成TFRecords文件
        type: TFRecord文件的存放路径 
            'single': 单标签
            'multi'： 多标签
        start, end: 保存图片的
        option: TFRecord文件保存的压缩格式
        """
        #待写 start end参数检查 但是超出index范围没关系 
        if type == 'single': #选择的是单标签的图片
            tfr_file = TS_TFRecords_File
            nls = self.train_single_labels
        else: # type == 'multi'
            tfr_file = TM_TFRecords_File
            nls = self.train_multi_labels

        trf_writer = tf.python_io.TFRecordWriter(tfr_file, options=option)

        nls = nls.sample(frac=1).reset_index(drop=True)    #打乱顺序 frac是要返回的比例 1=100%
        for index, (name, label) in nls[start:end].iterrows():
            img = read_image(name, 'train')
            img = self.__preprocessing(img) #裁剪 归一化等预处理
            img_shape = img.shape
            img = img.reshape(-1) #变成一维
            img_list = img.tolist() 
            feature_internal = {
                    'image_raw' : tf.train.Feature(float_list = tf.train.FloatList(value=img_list)), #内层feature编码方式
                    'img_shape' : tf.train.Feature(int64_list = tf.train.Int64List(value=img_shape)),
                    'label' : tf.train.Feature(int64_list = tf.train.Int64List(value=[int(label)]))     
                    }
            #使用tf.train.Example将features编码数据封装成特定的PB协议格式
            example = tf.train.Example(features=tf.train.Features(feature=feature_internal))
            #将序列化为字符串的example数据写入协议缓冲区
            print('No.{}: trf_writer写入{} label:{}'.format(index, name, label))
            trf_writer.write(example.SerializeToString())
        #关闭TFRecords文件操作接口    
        trf_writer.close()

    def get_dataset_from_TFRecoard(self, type, batch_size=None):
        """
        ##从tfr_files指定的TFRecords文件，初始化一个dataset
        :param 
        type: TFRecord文件的存放路径 
            'single': 单标签
            'multi'： 多标签            
        batch_size: 参数表示batch的消息 None不处理 
        :return: 
        ！可以考虑用slim模块实现
        """
        if type == 'single': #选择的是单标签的图片
            tfr_files = TS_TFRecords_File
        else: # type == 'multi'
            tfr_files = TM_TFRecords_File

        # 定义TFRecordDataset
        dataset = tf.data.TFRecordDataset(tfr_files)
        # Set the number of datapoints you want to load and shuffle 
        dataset = dataset.shuffle(buffer_size=1000)
        #执行解析函数 得到数据集    
        dataset = dataset.map(self.__parse_function)
        # 定义batchsize大小
        if batch_size:
            dataset = dataset.batch(batch_size)
        # 不加参数->无限重复数据集
        dataset = dataset.repeat()

        # Create an iterator
        iterator = dataset.make_one_shot_iterator()
        
        # Create your tf representation of the iterator
        image, label = iterator.get_next()

        return image, label

    def __preprocessing(self, img):
        """对图像数据进行预处理"""
        #其他处理。。。
        #resize crop变成H*W大小
        img = self.__resize(img)

        # img_norm = tf.image.per_image_standardization(timg) #tensorflow中对图像标准化预处理的API
        img_array = np.asarray(img, np.float32)

        img_norm = img_array/255.  #[0,1]归一化
        return img_norm

    def __resize(self, img):
        """对图像裁切 缩减"""
        H, W = X_shape[:2]   #输入模型需要的图片的高 宽
        h, w = img.size[::-1]  #原始图片的高 宽
        if h > w:  #如果原图宽比较小
            if w > W*1.1:  #宽比要求的尺寸高10%以上
                image = img.resize((int(W*1.1), int(h/w*W*1.1)), Image.ANTIALIAS)  #宽缩到110% 剩下的裁剪掉
            elif w < W:  #宽比要求的尺寸小
                image = img.resize((W, int(h*W/w)), Image.ANTIALIAS) #宽放大到需要尺寸 高同步放大
            else: image = img
        else: #原图高比较小
            if h > H*1.15:  #高 多15%以上
                image = img.resize((int(w/h*H*1.15), int(H*1.15)), Image.ANTIALIAS)
            elif h < H:
                image = img.resize((int(w*H/h), H), Image.ANTIALIAS)
            else: image = img
        #对resize好的图片 进行裁剪，满足尺寸H,W
        h, w = image.size[::-1] #resize后的尺寸
        top = int(0.2 * (h-H)) #上高裁切 取 多余的20%
        left = int(0.5 * (w-W)) #宽度多余的 左右各裁剪一半
        image = image.crop((left, top, left+W, top+H)) #四周裁剪 留下H*W区域
        return image

    def __parse_function(self, example_proto):
        """解析函数 
        :param example_proto: example序列化后的样本tf_serialized
        """
        features = {
            'image_raw' : tf.VarLenFeature(dtype=tf.float32),
            'img_shape' : tf.FixedLenFeature(shape=(3,), dtype=tf.int64),
            'label' : tf.FixedLenFeature(shape=(), dtype=tf.int64)
            }
        # 把序列化样本和解析字典送入函数里得到解析的样本
        parsed_example = tf.parse_single_example(example_proto, features) #返回字典
        # 解码 
        img = tf.sparse_tensor_to_dense(parsed_example['image_raw'], default_value=0) # 稀疏->密集表示
        img = tf.reshape(img, parsed_example['img_shape']) # 转换tensor形状
        # 如果使用dataset作为keras中，model.fit函数等的参数，则需要使用one_hot编码
        # 在tensorflow中，基本是不需要的，可以直接返回example['label']。
        # label = tf.one_hot(parsed_example['label'], self.train_single_num)
        # label = tf.cast(parsed_example['label'], tf.uint8)
        label = parsed_example['label']
        return img, label

    def _get_train_label(self):
        """获取所有的train label"""
        labels = pd.read_csv(Train_Label_Path)
        return labels

    def _get_train_img(self, start=0, end=100):
        """测试用 读图片
        return: Image格式
        """
        for i in range(start, end):
            (name, label) = self.train_single_labels.iloc[i]        
            img = read_image(name, 'train')
            yield(img, label)



In [6]:
"""****************************************"""
""" 模型                 """
"""****************************************"""

def Conv2D_BN(x, filters, kernel_size, padding='same', activation='relu'):
    """加BN层的卷积
    # Returns
        Output tensor after applying `Conv2D` and `BatchNormalization`
    """
    x = Conv2D(filters, kernel_size, padding=padding)(x)
    x = BatchNormalization()(x)
    if activation: 
        x = Activation(activation)(x)
    return x

def inception_module(x, params, concat_axis):
    """
    inception块
    param :
        x: 输入的张量，维度为（m, n_H, n_W, n_C）
        params: 每层输出通道数
    """
    (branch1, branch2, branch3, branch4) = params #每层输出通道数
    #inception module
    pathway1 = Conv2D_BN(x, branch1, (1,1))
    pathway2 = Conv2D_BN(x, branch2[0], (1,1))
    pathway2 = Conv2D_BN(pathway2, branch2[1], (3,3))
    pathway3 = Conv2D_BN(x, branch3[0], (1,1))
    pathway3 = Conv2D_BN(pathway3, branch3[1], (3,3))
    pathway3 = Conv2D_BN(pathway3, branch3[1], (3,3))
    pathway4 = MaxPool2D(pool_size=(3,3), strides=1, padding='same')(x)
    pathway4 = Conv2D_BN(pathway4, branch4, (1,1))
    return Concatenate(axis=concat_axis)([pathway1,pathway2,pathway3,pathway4])

def resdual_module(inputs, params, name):
    """
    三层的恒等残差块
    param :
        inputs: 输入的张量，维度为（m, n_H, n_W, n_C）
        params: 每层输出通道数

    """
    with tf.variable_scope(name):
        (branch1, branch2, branch3) = params #每层输出通道数
        x = Conv2D_BN(inputs, branch1, kernel_size=(1,1), padding='valid', activation='relu')
        x = Conv2D_BN(x, branch2, kernel_size=(3,3), padding='same', activation='relu')
        x = Conv2D_BN(x, branch3, kernel_size=(1,1), padding='valid', activation=None)
        x = Add()([x, inputs]) 
        x = Activation('relu')(x)
    return x
 
#load VGG19 模型，不包括全连接层
base_model = VGG19(weights = 'imagenet', 
           include_top = False, 
           input_tensor = Input(shape=X_shape)
           )
#冻结
for layer in base_model.layers:  
    layer.trainable = False 

model = Model(base_model.inputs, base_model.outputs)

# x = Conv2D_BN(inputs, 64, (3,3), padding='same')
# x = MaxPool2D()(x)
 
# x = resdual_module(x, (16,16,64), name="Res_block1")
# x = resdual_module(x, (16,16,64), name="Res_block2")
# x = resdual_module(x, (16,16,64), name="Res_block3")
# x = resdual_module(x, (16,16,64), name="Res_block3")

# x = inception_module(x, params=(32,(32,64),(24,48),16), concat_axis=-1)
# x = MaxPool2D()(x)

# x = Flatten()(x)
# x = Dense(128)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
# x = Dropout(0.5)(x)
# predictions = Dense(10, activation='softmax')(x)
# model = Model(inputs, predictions,)

In [7]:
# model.summary()
model.compile(optimizer='adam',
       loss='categorical_crossentropy'
       )

In [0]:
dg = DatasetGenerator()
#判断文件sfou是否存在（colab会自动删除）
if not os.path.exists(TS_TFRecords_File): 
    dg.write2TFRecoard('single',0,255) #写tfrecoard


In [9]:
img, label = dg.get_dataset_from_TFRecoard(type='single')

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [0]:
# hist = model.fit()

In [11]:
# model.evaluate_generator(img,)
# model.predict_generator(img, steps=100)

TypeError: ignored

In [0]:
###### 以下测试 /////////////////////////////////////////////
sess = tf.InteractiveSession() 
sess.run(tf.global_variables_initializer())

In [0]:
dg.write2TFRecoard(type='single', ) #写入到tfrecoard

刚开始H W: 396 396
resize 后 H W: 500 500
left, top, left+W, top+H 0 0 500 500
No.0: trf_writer写入4fc2970954df40f19e62b655e92d77df.jpg label:18
刚开始H W: 250 268
resize 后 H W: 500 536
left, top, left+W, top+H 18 0 518 500
No.1: trf_writer写入aa8067f6a02e412290ad660800e34980.jpg label:2
刚开始H W: 1280 720
resize 后 H W: 977 550
left, top, left+W, top+H 25 95 525 595
No.2: trf_writer写入0817c1defbeb4419b70fe138b7f9b64e.png label:8
刚开始H W: 946 945
resize 后 H W: 550 550
left, top, left+W, top+H 25 10 525 510
No.3: trf_writer写入4afa723d870843d9a7a568c256a88083.jpg label:25
刚开始H W: 789 792
resize 后 H W: 575 577
left, top, left+W, top+H 38 15 538 515
No.4: trf_writer写入b9a9a0e7dd4c42118817fa4bbcdd4eb2.jpg label:2
刚开始H W: 792 792
resize 后 H W: 575 575
left, top, left+W, top+H 37 15 537 515
No.5: trf_writer写入d5c10d49dccd430199839bb5851b5623.jpg label:25
刚开始H W: 792 792
resize 后 H W: 575 575
left, top, left+W, top+H 37 15 537 515
No.6: trf_writer写入6cd5cecab3974f2b86fd06390cb63d65.jpg label:7
刚开始H W: 1656 2484
r

In [0]:
for i in range(11):
    print('No.{} image shape:'.format(i+1), img.shape)
    plt.imshow(img.eval())
    plt.show()

In [0]:
i = 1
for timg, l in dg._get_train_img(): #取测试图片
    i += 1
#     print(l)
#     plt.imshow(timg), timg.size
#     plt.show()    
#     r = dg._resize(timg)
#     plt.imshow(r), r.size 
#     plt.show()
    if i > 5: break